# Tietojenkäsittelytieteen opastus VS Code 

## Osa 1. Esivaatimukset:
- Visual Studio Code
- Laajennukset:
    - Python extension for VS Code
    - Jupyter extension for VS Code
- Miniconda tai Anaconda

## Osa 2. Määritä data science ympäristö

### Asennus

1. Kloonaa repositorio:

        git clone https://github.com/yourusername/yourproject.git

2. Siirry projektin hakemistoon:

        cd data-science-tutorial
   
3. Määritä Conda-ympäristö käyttämällä tämän reposta löytyvää yml-tiedostoa:

        conda env create -f environment.yml

4. Aktivoi ympäristö:

        conda activate myenv

## Osa 3. Valmistele data

- Käsittele ja puhdista data
- Tarkista korrelaatiot
- Valmistelun jälkeen sinulla on datasetti, jota voidaan käyttää mallin kouluttamiseen
   - tässä tapauksessa valitsimme seuraavat muuttujat: 'sex', 'pclass', 'age', 'relatives', 'fare', 'survived'


In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('titanic3.csv')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=5, figsize=(30,5))
sns.violinplot(x="survived", y="age", hue="sex", data=data, ax=axs[0])
sns.pointplot(x="sibsp", y="survived", hue="sex", data=data, ax=axs[1])
sns.pointplot(x="parch", y="survived", hue="sex", data=data, ax=axs[2])
sns.pointplot(x="pclass", y="survived", hue="sex", data=data, ax=axs[3])
sns.violinplot(x="survived", y="fare", hue="sex", data=data, ax=axs[4])

In [ ]:
data.replace({'male': 1, 'female': 0}, inplace=True)

In [ ]:
data.corr(numeric_only=True).abs()[["survived"]]

In [ ]:
data['relatives'] = data.apply (lambda row: int((row['sibsp'] + row['parch']) > 0), axis=1)
corr_with_survived = data.corr(numeric_only=True).abs()[["survived"]]
print(corr_with_survived)

In [ ]:
# use this if you want to choose specific columns for example to skip body that is not needed.
'''
data['relatives'] = data.apply (lambda row: int((row['sibsp'] + row['parch']) > 0), axis=1)

columns_to_check = ['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare', 'relatives']

corr_matrix = data[columns_to_check].corr()

corr_with_survived = corr_matrix[['survived']].abs()

print(corr_with_survived)
'''

In [7]:
data = data[['sex', 'pclass','age','relatives','fare','survived']].dropna()

In [8]:
# optional: use this to run the python file version.

# %run script.py

## Osa 4. Kouluta ja arvioi malli
   Tässä osiossa käytä scikit-learn-kirjastoa (se tarjoaa hyödyllisiä apufunktioita) datasetin käsittelyyn, luokitellaksesi mallin, joka ennustaa selviytymistä laivalla. Käytä sen jälkeen mallia testidatan kanssa arvioidaksesi sen tarkkuuden.

### 4.1 Jaa datasetti koulutus- ja validointidataan
   Yleinen ensimmäinen askel mallin kouluttamisessa on jakaa datasetti koulutus- ja validointidataan. Tämä mahdollistaa osan datasta käyttämisen mallin kouluttamiseen ja osan mallin testaamiseen. Jos käyttäisit koko dataa mallin kouluttamiseen, sinulla ei olisi keinoa arvioida, kuinka hyvin se todellisuudessa suoriutuisi sellaista dataa vastaan, jota malli ei ole vielä nähnyt. Scikit-learn-kirjaston etuna on se, että se tarjoaa metodin datasetin jakamiseen koulutus- ja testidataan. Lisää ja suorita muistikirjaan seuraava koodi, jolla jaat datan.

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[['sex','pclass','age','relatives','fare']], data.survived, test_size=0.2, random_state=0)

### 4.2 Normalisoi syötteet
   Seuraavaksi normalisoi syötteet siten, että kaikkia ominaisuuksia käsitellään tasapuolisesti. Esimerkiksi datasetissä ikäarvot vaihtelevat noin 0-100 välillä, kun taas sukupuoli on vain 1 tai 0. Normalisoimalla kaikki muuttujat voit varmistaa, että arvojen vaihteluvälit ovat yhtenevät. Käytä seuraavaa koodia uudessa koodisolussa syötearvojen skaalaamiseksi.

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

### 4.3 Valitse, luo ja kouluta algoritmi
Koneoppimisalgoritmeja on monia, joista voit valita mallintaaksesi dataa. Scikit-learn-kirjasto tukee monia niistä ja tarjoaa kaavion, joka auttaa valitsemaan oikean algoritmin sinun tilanteeseesi. Tällä kertaa käytä Naïve Bayes -algoritmia, joka on yleinen algoritmi luokitteluongelmiin. Lisää solu, jossa on seuraava koodi, algoritmin luomiseksi ja kouluttamiseksi.


In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

### 4.4 Kokeile koulutettua mallia testidatalla

Koulutetun mallin avulla voit nyt kokeilla sitä sellaista testidataa vastaan, jota ei käytetty koulutuksessa. Lisää ja suorita seuraava koodi ennustaaksesi testidatan tulokset ja laskeaksesi mallin tarkkuuden.

In [ ]:
from sklearn import metrics
predict_test = model.predict(X_test)
print(metrics.accuracy_score(y_test, predict_test))

## Osa 5 (Valinnainen) Käytä neuroverkkoa

Neuroverkko on malli, joka käyttää painotuksia ja aktivointifunktioita, mallintaen ihmisen hermosolujen toimintaa, päätelläkseen lopputuloksen annettujen syötteiden perusteella. Toisin kuin aiemmin käsitellyt koneoppimisalgoritmit, neuroverkot ovat syväoppimisen muoto, jossa sinun ei tarvitse tietää etukäteen ideaalista algoritmia ongelmanratkaisuusi. Niitä voidaan käyttää monissa eri tilanteissa, ja luokittelu on yksi näistä. Tässä osiossa käytät Keras -kirjastoa yhdessä TensorFlow kanssa rakentaaksesi neuroverkon ja tutkiaksesi, kuinka se käsittelee Titanic-datasettiä.

### 5.1 Tuo kirjastot ja luo malli

Ensimmäinen askel on tarvittavien kirjastojen tuominen ja mallin luominen. Tässä tapauksessa käytät Sequential -neuroverkkoa, joka on kerroksittainen neuroverkko, jossa useat kerrokset syöttävät tietoa toisiinsa peräkkäin.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()


### 5.2 Luo neuroverkon kerrokset

Mallin määrittelyn jälkeen seuraava askel on lisätä neuroverkon kerrokset. Pidetään asiat toistaiseksi yksinkertaisina ja käytetään vain kolmea kerrosta. Lisää seuraava koodi luodaksesi neuroverkon kerrokset.

In [14]:
model.add(Dense(5, kernel_initializer = 'uniform', activation = 'relu', input_dim = 5))
model.add(Dense(5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))

- Ensimmäisellä kerroksella asetetaan ulottuvuudeksi 5, koska sinulla on viisi syötettä: sukupuoli matkustusluokka, ikä, sukulaiset ja lipun hinta.
- Viimeisen kerroksen täytyy tuottaa ulostuloksi 1, koska haluat yksidimensionaalisen ulostulon, joka ilmaisee, selviääkö matkustaja.
- Keskimmäinen kerros pidettiin yksinkertaisuuden vuoksi arvossa 5, vaikka se arvo voisi olla erilainen.

Rectified linear unit (relu) -aktivointifunktiota käytetään hyvänä yleisenä aktivointifunktiona kahdessa ensimmäisessä kerroksessa, kun taas sigmoid-aktivointifunktio on tarpeen viimeiselle kerrokselle, koska haluamasi ulostulo (matkustajan selviytyminen tai ei) täytyy skaalata alueelle 0-1 (todennäköisyys matkustajan selviytymiselle).

Voit myös tarkastella rakentamasi mallin yhteenvetoa tällä koodirivillä:

In [ ]:
model.summary()

### 5.3 Rakenna ja kouluta

Kun malli on luotu, se täytyy koota. Osana tätä prosessia sinun täytyy määritellä, millaista optimoijaa käytetään, miten häviö (loss) lasketaan, ja mitä metriikkaa tulisi optimoida. Lisää seuraava koodi mallin rakentamiseksi ja kouluttamiseksi. Huomaat, että koulutuksen jälkeen tarkkuus on noin 61%.

Huom: Tämän vaiheen suorittaminen voi kestää koneestasi riippuen muutamasta sekunnista muutamaan minuuttiin.

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=50)

### 5.4 Testaa luotua mallia testidatalla

Nyt kun malli on rakennettu ja koulutettu, voimme nähdä, miten se toimii testidataa vastaan.

In [ ]:
y_pred = np.rint(model.predict(X_test).flatten())
print(metrics.accuracy_score(y_test, y_pred))

Samoin kuin koulutusvaiheessa, huomaat nyt saavuttavasi 79 % tarkkuuden matkustajien selviytymisen ennustamisessa. Käyttäen tätä yksinkertaista neuroverkkoa tulos on parempi kuin aiemmin kokeillun Naive Bayes -luokittelijan 75 % tarkkuus.


## Seuraavat askeleet

Nyt kun olet perehtynyt koneoppimisen perusteisiin Visual Studio Code -ympäristössä, tässä on joitain muita Microsoftin resursseja ja opetusohjelmia, joita voit tutkia.

- [Data Science profiilimalli](https://code.visualstudio.com/docs/editor/profiles#_data-science-profile-template) – Luo uusi [profiili](https://code.visualstudio.com/docs/editor/profiles), jossa on valikoitu joukko laajennuksia, asetuksia ja koodinpätkiä.
- Lue lisää [Jupyter-muistikirjojen käyttämisestä Visual Studio Code -ympäristössä](https://youtu.be/FSdIoJdSnig) (video).
- [Aloita Azure Machine Learningin käyttö Visual Studio Code](https://learn.microsoft.com/azure/machine-learning/how-to-setup-vs-code) käyttöönottoa ja mallin optimointia varten hyödyntämällä Azurea.
- Löydä lisää tutkittavaa dataa [Azure Open Data Sets](https://azure.microsoft.com/services/open-datasets/) -palvelusta.